In [1]:
import pandas as pd 
import requests

In [2]:
from bitrix24 import Bitrix24
bx24 = Bitrix24('https://b24-da84sm.bitrix24.com.br/rest/163/7vn3yt69x1bw13gk')
response =  bx24.callMethod('user.get')


In [20]:
departments = bx24.callMethod('department.get')


In [17]:
usuarios = pd.DataFrame(response)
usuarios = usuarios[['ID', 'NAME', 'ACTIVE','LAST_NAME','DATE_REGISTER', 'UF_DEPARTMENT']]


In [18]:
usuarios = usuarios.explode('UF_DEPARTMENT')
usuarios.columns = ['ID', 'NAME', 'ACTIVE','LAST_NAME','DATE_REGISTER', 'DEPARTAMENTO_BITRIX']


In [21]:
departamentos = pd.DataFrame(departments)
departamentos

,ID,NAME,SORT,UF_HEAD,PARENT
0,1,Droga Leste,500,13,NaN
1,35,Adm Pessoal,500,45,1
2,256,Controladoria & Risco,500,131,1
3,3,Contábil / Fiscal,500,81,256
4,270,Planejamento e Controles,500,NaN,256
5,268,Qualidade e Processos,500,1192,256
6,272,zPostos,500,205,256
7,76,Departamento Juridico,500,121,1
8,78,Escritório Blaumer,500,NaN,76
9,80,Escritório Nogueira Leite,500,NaN,76


In [6]:
mapa = {
15:     222,
17:     222,
19:     222,
59:     222,
224:    222,
234:    222,
222:    222,
51:     51,
29:     51,
86:     51,
88:     51,
33  :   254,
37  :   254,
220 :   254,
226 :   254,
258 :   258,
31  :   258,
43  :   258,
240 :   258,
256:    256,
3   :   256,
270 :   256,
268 :   256,
272 :   256,
240	:	258,
246	:	240,
252	:	240,
242	:	240,
248	:	240,
244	:	240,
250	:	240,
31	:	258,
43	:	258,
35  :   35,
21  :   21,
236	:   236,
260	:   236,
274	:   236,
262 :   262,
74  :	262,
57  :	262,
254	:	254,
226	:	254,
33	:	254,
280	:	254,
278:    254,
7	:   7,
264	:   7,
298 :   7,
238	:   7,
158	:   158,
1:      1,
76	:   1,
78	:   76,
80	:   76,
258	: 1,
220	: 254,
266	: 258,
240	: 258
}

In [7]:
usuarios['DEPARTAMENTO'] = usuarios['DEPARTAMENTO_BITRIX'].map(mapa)

In [8]:
usuarios = usuarios.drop_duplicates(subset= ['ID', 'NAME'],keep = 'first')

In [9]:
ativo = {
    True: 'sim',
    False: 'não'
}

usuarios['ativo'] = usuarios['ACTIVE'].map(ativo)

In [10]:
# usuarios = usuarios.drop('ACTIVE', axis=1)
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 451 entries, 0 to 450
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   451 non-null    object 
 1   NAME                 449 non-null    object 
 2   ACTIVE               451 non-null    bool   
 3   LAST_NAME            449 non-null    object 
 4   DATE_REGISTER        451 non-null    object 
 5   DEPARTAMENTO_BITRIX  451 non-null    object 
 6   DEPARTAMENTO         450 non-null    float64
 7   ativo                451 non-null    object 
dtypes: bool(1), float64(1), object(6)
memory usage: 28.6+ KB


In [16]:
from sqlalchemy import Column, Integer, DATETIME, create_engine, Table, select, MetaData, URL, and_
from datetime import datetime, timedelta
from sqlalchemy.orm import sessionmaker, declarative_base

server_name = r'suportedl.no-ip.org\nero'
database_name = 'BILLS'
user ='CASSIO'
password = '@BLADE1212'
drive = 'ODBC Driver 17 For SQL Server'

connection_string = f"DRIVER={drive};SERVER={server_name};DATABASE={database_name};UID={user};PWD={password}"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

usuarios.to_sql('COLABORADORES', con=engine, if_exists='append', index= False)


189

In [ ]:
resultados = pd.read_excel('Results.xlsx')
resultados



0      False
0      False
0      False
0      False
0      False
       ...  
446    False
447    False
448    False
449    False
450    False
Length: 500, dtype: bool